In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
inputs=keras.Input(shape=(784,),name="digits")
x=layers.Dense(64,activation='relu',name='dense_1')(inputs)
x=layers.Dense(64,activation='relu',name='dense_2')(x)
outputs=layers.Dense(10,activation="softmax",name="predictions")(x)

model=keras.Model(inputs=inputs,outputs=outputs)

In [5]:
(x_train,y_train),(x_test,y_test)=keras.datasets.mnist.load_data()
x_train=x_train.reshape(60000,784).astype("float32")/255
x_test=x_test.reshape(10000,784).astype("float32")/255

y_train=y_train.astype("float32")
y_test=y_test.astype("float32")

x_val=x_train[-10000:]
y_val=y_train[-10000:]
x_train=x_train[:-10000]
y_train=y_train[:-10000]

11493376/11490434 [==============================] - 1s 0us/step


In [6]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalCrossentropy()]
)

In [7]:
history=model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=2,
    validation_data=(x_val,y_val)
)

Epoch 1/2
782/782 [==============================] - 2s 2ms/step - loss: 0.3395 - sparse_categorical_crossentropy: 0.3395 - val_loss: 0.1801 - val_sparse_categorical_crossentropy: 0.1801
Epoch 2/2
782/782 [==============================] - 2s 2ms/step - loss: 0.1587 - sparse_categorical_crossentropy: 0.1587 - val_loss: 0.1318 - val_sparse_categorical_crossentropy: 0.1318


In [8]:
history.history

{'loss': [0.33953961730003357, 0.15870992839336395],
 'sparse_categorical_crossentropy': [0.33953961730003357, 0.15870992839336395],
 'val_loss': [0.1800721138715744, 0.13176694512367249],
 'val_sparse_categorical_crossentropy': [0.1800721138715744,
  0.13176694512367249]}

In [9]:
results=model.evaluate(x_test,y_test,batch_size=128)
print("test loss,train_loss",results)


79/79 [==============================] - 0s 1ms/step - loss: 0.1414 - sparse_categorical_crossentropy: 0.1414
test loss,train_loss [0.14143158495426178, 0.14143158495426178]


In [10]:
print("Generate predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape:", predictions.shape)

Generate predictions for 3 samples
predictions shape: (3, 10)


In [12]:
def get_uncompiles_model():
  inputs=keras.Input(shape=(784,),name="digits")
  x=layers.Dense(64,activation='relu',name='dense_1')(inputs)
  x=layers.Dense(64,activation='relu',name='dense_2')(x)
  outputs=layers.Dense(10,activation="softmax",name="predictions")(x)
  model=keras.Model(inputs=inputs,outputs=outputs)
  return model

def get_compiled_model():
  model=get_uncompiles_model()
  model.compile(
      optimizer='rmsprop',
      loss='sparse_categorical_crossentropy',
      metrics=['sparse_categorical_crossentropy']
  )
  return model


In [13]:
model = get_compiled_model()

# First, let's create a training Dataset instance.
# For the sake of our example, we'll use the same MNIST data as before.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# Shuffle and slice the dataset.
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

# Now we get a test dataset.
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(64)

# Since the dataset already takes care of batching,
# we don't pass a `batch_size` argument.
model.fit(train_dataset, epochs=3)

# You can also evaluate or predict on a dataset.
print("Evaluate")
result = model.evaluate(test_dataset)
dict(zip(model.metrics_names, result))

Epoch 1/3
782/782 [==============================] - 2s 2ms/step - loss: 0.3490 - sparse_categorical_crossentropy: 0.3490
Epoch 2/3
782/782 [==============================] - 2s 2ms/step - loss: 0.1588 - sparse_categorical_crossentropy: 0.1588
Epoch 3/3
782/782 [==============================] - 2s 2ms/step - loss: 0.1155 - sparse_categorical_crossentropy: 0.1155
Evaluate
157/157 [==============================] - 0s 1ms/step - loss: 0.1164 - sparse_categorical_crossentropy: 0.1164


{'loss': 0.1163959726691246,
 'sparse_categorical_crossentropy': 0.1163959726691246}